In [1]:
import findspark
import os
import gc
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
findspark.init(SPARK_HOME)

import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
import numpy as np
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import size, array_union,flatten,array_sort,coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
import geofeather
from pyspark.storagelevel import StorageLevel

from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
#os.environ['PYSPARK_PYTHON'] = "/home/qiany/.conda/envs/py37/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [2]:
tin_file = input("Here is a programe to extract boundary relations, please input the absolute or relative path to your .ts file:")

# get the directory, basename of the input file
print("\n********************")
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or Tri_data(1) or Tetra_data (2):") or "2"

if directory_type == '0':
    directory = 'hdfs_data'
elif directory_type == '1':
    directory = 'Tri_data'
else:
    directory = 'Tetra_data'
    
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)

print("\n********************")
print("This is a TIN file in \"%s\" format" % tin_extension)

filtra = 'yes'

# allocate the number of executors, the number of cores per executor, and the amount of memory per executor
Num_executor = '64'
Num_core_per_executor = '5'
Memory_executor = '64g'
MemoryOverhead_executor = '8g'

# allocate the number of cores for the driver node
Num_core_per_driver = '5'
Memory_driver = '64g'
MemoryOverhead_driver = '32g'

# the default number of shuffle partitions
Num_shuffle_partitions = input("spark.sql.shuffle.partitions:")


********************
tin_directory:  /home/qiany/yuehui/pyspark/Tetra_mesh/data
tin_basename:  Lander_big.ts
tin_filename:  Lander_big
tin_extension:  .ts

********************
This is a TIN file in ".ts" format


In [3]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

spark_app_name = "TopoRela_CritPts_Vanilla_Spark_" + tin_filename + '_' + date_name + '_' + hour_name
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", KryoSerializer.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '400') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config("spark.sql.objectHashAggregate.sortBased.fallbackThreshold", "-1") \
.config('spark.yarn.dist.archives', '/local/data/yuehui/py37.tar.gz#environment') \
.config("spark.python.profile", "true") \
.config("spark.eventLog.enabled", "true") \
.config("spark.eventLog.logStageExecutorMetrics", "true") \
.getOrCreate()

spark_app_name: TopoRela_CritPts_separate_TetraMesh_Lander_big_06162024_0157


### read input files

In [4]:
# read_tetra_order() is a function used to read tetrahedron from a csv file
def read_tetra_order(hdfs_tetra_origin):
    '''
    this function has two input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a tetrahedra file
    tin_filename: a string denoting the file name of a tetrahedra extension, e.g., 827_monviso
    '''
        
    schema_tetra_origin = StructType([ \
        StructField("tetra_order",IntegerType(),True), \
        StructField("r1",IntegerType(),True), \
        StructField("r2",IntegerType(),True), \
        StructField("r3",IntegerType(),True), \
        StructField("r4",IntegerType(),True), \
        StructField("r1_ele",FloatType(),True), \
        StructField("r2_ele",FloatType(),True), \
        StructField("r3_ele",FloatType(),True), \
        StructField("r4_ele",FloatType(),True) \
      ])

    df_tetra_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tetra_origin)\
          .load(hdfs_tetra_origin)
        
    return df_tetra_origin


# read tetrahedra
hdfs_tetra_origin = directory + "/" + tin_filename + '_filtra_tetra_sort.csv'

df_tetra_order = read_tetra_order(hdfs_tetra_origin)
df_tetra_order.printSchema()

Number of partitions for df_tetra_origin: 4959
root
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r4: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- r4_ele: float (nullable = true)



### globally obtain VT relation

In [5]:
# function to get VT directly from DF_T
def get_VT(df_tetra_order):
    df_tetra_order = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False))
    df_VT_init_1 = df_tetra_order.select("r1","tetra")
    df_VT_init_2 = df_tetra_order.select("r2","tetra")
    df_VT_init_3 = df_tetra_order.select("r3","tetra")
    df_VT_init_4 = df_tetra_order.select("r4","tetra")
    
    df_VT_union12 = df_VT_init_1.union(df_VT_init_2)
    df_VT_union123 = df_VT_union12.union(df_VT_init_3)
    df_VT_union1234 = df_VT_union123.union(df_VT_init_4)
    
    df_VT = df_VT_union1234.groupBy('r1').agg(collect_list('tetra').alias('VT'))
    df_VT = df_VT.withColumnRenamed('r1', 'Ver')
    
    return df_VT

In [6]:
df_VT = get_VT(df_tetra_order)

df_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [7]:
t0 = time.time()

num_row = df_VT.count()

t1 = time.time()
t_count_VT = t1 - t0
print("Time cost:", t_count_VT)
print("number of rows:", num_row)

Time cost: 501.4889419078827
number of rows: 1792989718


### globally obtain VF relation

In [8]:
# function to get VF from DF_T
def get_VF_init(df_tetra_order):
    df_VF_1 = df_tetra_order.withColumn("f1", sort_array(F.array("r1", "r2", "r3"), False)).withColumn("f2", sort_array(F.array("r1", "r2", "r4"), False)).withColumn("f3", sort_array(F.array("r1", "r3", "r4"), False)).drop('r2', 'r3', 'r4')
    df_VF_2 = df_tetra_order.withColumn("f1", sort_array(F.array("r2", "r1", "r3"), False)).withColumn("f2", sort_array(F.array("r2", "r1", "r4"), False)).withColumn("f3", sort_array(F.array("r2", "r3", "r4"), False)).drop('r1', 'r3', 'r4')
    df_VF_3 = df_tetra_order.withColumn("f1", sort_array(F.array("r3", "r1", "r2"), False)).withColumn("f2", sort_array(F.array("r3", "r1", "r4"), False)).withColumn("f3", sort_array(F.array("r3", "r2", "r4"), False)).drop('r1', 'r2', 'r4')
    df_VF_4 = df_tetra_order.withColumn("f1", sort_array(F.array("r4", "r1", "r2"), False)).withColumn("f2", sort_array(F.array("r4", "r1", "r3"), False)).withColumn("f3", sort_array(F.array("r4", "r2", "r3"), False)).drop('r1', 'r2', 'r3')
    
    df_VF_union12 = df_VF_1.union(df_VF_2)
    df_VF_union123 = df_VF_union12.union(df_VF_3)
    df_VF_union1234 = df_VF_union123.union(df_VF_4)
    
    df_VF_init = df_VF_union1234.groupBy('r1').agg(collect_set('f1').alias('multi_f1'), collect_set('f2').alias('multi_f2'), collect_set('f3').alias('multi_f3'))
    return df_VF_init

df_VF_init = get_VF_init(df_tetra_order)

# df_VE_init.cache()

df_VF_init.printSchema()

root
 |-- r1: integer (nullable = true)
 |-- multi_f1: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_f2: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_f3: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [9]:
# obtain VF relation
def get_VF(multi_f1, multi_f2, multi_f3):
# get_VF is used to obtain a complete VF relation from the partial VF relations
# multi_f1: partial VF relation
# multi_f2: partial VF relation
# multi_f3: partial VF relation

    faces = set()
    for f in multi_f1:
        faces.add(tuple(f))
        
    for f in multi_f2:
        faces.add(tuple(f))
        
    for f in multi_f3:
        faces.add(tuple(f))
    
    faces_list = sorted(faces) # save more time when using list(faces)
    
    return faces_list

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_VF_udf = udf(get_VF, ArrayType(ArrayType(IntegerType())))

df_VF = df_VF_init.withColumn("VF", get_VF_udf(df_VF_init.multi_f1, df_VF_init.multi_f2, df_VF_init.multi_f3)).drop('multi_f1', 'multi_f2', 'multi_f3')
df_VF = df_VF.withColumnRenamed('r1', 'Ver')

df_VF.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [10]:
t0 = time.time()

num_row = df_VF.count()

t1 = time.time()
t_count_VF = t1 - t0
print("Time cost:", t_count_VF)
print("number of rows:", num_row)

Time cost: 460.74542474746704
number of rows: 1792989718


### globally obtain VV relation

In [11]:
# function to get VV using groupby
def get_VV(df_tetra_order):
    df_VV_init_1 = df_tetra_order.select("r1", "r2", "r3", "r4")
    df_VV_init_2 = df_tetra_order.select("r2", "r1", "r3", "r4")
    df_VV_init_3 = df_tetra_order.select("r3", "r1", "r2", "r4")
    df_VV_init_4 = df_tetra_order.select("r4", "r1", "r2", "r3")
    
    df_VV_union1234 = df_VV_init_1.union(df_VV_init_2).union(df_VV_init_3).union(df_VV_init_4)
    
    df_VV_gp = df_VV_union1234.groupBy('r1').agg(collect_set('r2').alias('multi_r2_set'),collect_set('r3').alias('multi_r3_set'),collect_set('r4').alias('multi_r4_set'))
    
    return df_VV_gp

In [12]:
df_VV_gp = get_VV(df_tetra_order)

df_VV_gp.printSchema()

root
 |-- r1: integer (nullable = true)
 |-- multi_r2_set: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- multi_r3_set: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- multi_r4_set: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [13]:
# obtain VV relation
def get_multi_pt_order(pt_list1, pt_list2, pt_list3):
# get_multi_pt_index is used to obtain the adjacent vertexes index, including the vertex itself
# pt_list1: partial adjacent vertex indexes of join result 1
# pt_list2: partial adjacent vertex indexes of join result 2
# pt_list3: partial adjacent vertex indexes of join result 3
    pt_init = pt_list1 + pt_list2 + pt_list3
    pt_set = set(pt_init)
    
    pt_list = sorted(pt_set) # sorted(pt_set, reverse=False), False in ascending order while True in descending order
    
    return pt_list # directly return a list, e.g., [0, 1, 5, 6], the length of returned column list will be 12, incluing "[" and empty char

get_multi_pt_order_udf = udf(get_multi_pt_order, ArrayType(IntegerType()))

In [14]:
df_VV = df_VV_gp.withColumn("VV", get_multi_pt_order_udf(df_VV_gp.multi_r2_set, df_VV_gp.multi_r3_set, df_VV_gp.multi_r4_set)).drop('multi_r2_set', 'multi_r3_set', 'multi_r4_set')

df_VV = df_VV.withColumnRenamed('r1', 'Ver')

df_VV.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [15]:
t0 = time.time()

num_row = df_VV.count()

t1 = time.time()
t_count_VV = t1 - t0
print("Time cost:", t_count_VV)
print("number of rows:", num_row)

Time cost: 455.9120762348175
number of rows: 1792989718


### globally obtain TE relation

In [16]:
# function to get TE relation
def get_TE_from_T(df_tetra_order):
    df_TE = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False)).withColumn("e1", sort_array(F.array("r1", "r2",), False)).withColumn("e2", sort_array(F.array("r1", "r3",), False)).withColumn("e3", sort_array(F.array("r1", "r4",), False)).withColumn("e4", sort_array(F.array("r2", "r3",), False)).withColumn("e5", sort_array(F.array("r2", "r4",), False)).withColumn("e6", sort_array(F.array("r3", "r4",), False))
    df_TE = df_TE.select('tetra', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6')
    
    return df_TE

In [17]:
# df_TE.persist(StorageLevel.MEMORY_ONLY)

df_TE = get_TE_from_T(df_tetra_order)

df_TE.printSchema()

root
 |-- tetra: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e1: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e2: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e3: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e4: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e5: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e6: array (nullable = false)
 |    |-- element: integer (containsNull = true)



##### get TE where tetrahedra incident to the same vertex

In [18]:
df_VT_expl = df_VT.select("Ver", explode("VT").alias("single_tetra"))

df_TE_expected_init = df_VT_expl.join(df_TE, df_VT_expl.single_tetra == df_TE.tetra).drop('single_tetra')
df_TE_expected_init.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- tetra: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e1: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e2: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e3: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e4: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e5: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e6: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [19]:
def union_TE(tetra, e1, e2, e3, e4, e5, e6):
    # TE_dict = {}
    # TE_dict[tuple(tetra)] = [e1, e2, e3, e4, e5, e6]
    TE_ary = [[tetra], [e1, e2, e3, e4, e5, e6]]
    
    return TE_ary

union_TE_udf = udf(union_TE, ArrayType(ArrayType(ArrayType(IntegerType()))))

In [20]:
df_TE_expected_udf = df_TE_expected_init.withColumn("TE_expected", union_TE_udf(df_TE_expected_init.tetra, df_TE_expected_init.e1, df_TE_expected_init.e2, df_TE_expected_init.e3, df_TE_expected_init.e4, df_TE_expected_init.e5, df_TE_expected_init.e6))
df_TE_expected_udf = df_TE_expected_udf.select("Ver", "TE_expected")

df_TE_expected = df_TE_expected_udf.groupby("Ver").agg(collect_list('TE_expected').alias('TE_final'))
df_TE_expected.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- TE_final: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: integer (containsNull = true)



In [21]:
t0 = time.time()

num_row = df_TE_expected.count()

t1 = time.time()
t_count_TE_expected = t1 - t0
print("Time cost:", t_count_TE_expected)
print("number of rows:", num_row)

Time cost: 2889.5364854335785
number of rows: 1792989718


### concatenate the relations in one DataFrame

In [22]:
df_VT_VF = df_VT.join(df_VF, df_VT.Ver == df_VF.Ver)
df_VT_VF = df_VT_VF.select(df_VT.Ver, "VT", "VF")

df_VT_VF.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [23]:
df_VT_VF_VV = df_VT_VF.join(df_VV, df_VT_VF.Ver == df_VV.Ver)
df_VT_VF_VV = df_VT_VF_VV.select(df_VT_VF.Ver, "VT", "VF", "VV")

df_VT_VF_VV.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [24]:
df_VT_VF_VV_TE = df_VT_VF_VV.join(df_TE_expected, df_VT_VF_VV.Ver == df_TE_expected.Ver)
df_VT_VF_VV_TE = df_VT_VF_VV_TE.select(df_VT_VF_VV.Ver, "VT", "VF", "VV", "TE_final")

df_VT_VF_VV_TE.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- TE_final: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: integer (containsNull = true)



In [25]:
def bool_boundary(VT, VF):
    # check if Ver is a boundary vertex or internal vertex
    if 3*len(VT) == 2*len(VF):
        # this is an internal vertex
        return False
    else:
        return True

bool_boundary_udf = udf(bool_boundary, BooleanType())

df_Crit_rela = df_VT_VF_VV_TE.withColumn("BoundaryOrNot", bool_boundary_udf(df_VT_VF_VV_TE.VT, df_VT_VF_VV_TE.VF))
df_Crit_rela.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- TE_final: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |-- BoundaryOrNot: boolean (nullable = true)



In [26]:
t0 = time.time()

num_row = df_Crit_rela.count()

t1 = time.time()
t_count_Crit_rela = t1 - t0
print("Time cost:", t_count_Crit_rela)
print("number of rows:", num_row)

Time cost: 3812.731728553772
number of rows: 1792989718


### extracting critical points

In [27]:
# define UnionFind class outside the UDF
# the UnionFind class is an implementation of Union-Find algorithm
class UnionFind:
    def __init__(self, size):
        self.root = list(range(size))
        self.rank = [1] * size

    def find(self, x):
        if self.root[x] != x:
            self.root[x] = self.find(self.root[x])
        return self.root[x]

    def union(self, x, y):
        rootX = self.find(x)
        rootY = self.find(y)
        if rootX != rootY:
            if self.rank[rootX] > self.rank[rootY]:
                self.root[rootY] = rootX
            elif self.rank[rootX] < self.rank[rootY]:
                self.root[rootX] = rootY
            else:
                self.root[rootY] = rootX
                self.rank[rootX] += 1

In [28]:
# define an UDF to extracting critical vertices
def compute_crit_pts(Ver, VV, TE):
    # store VV in two groups, one group for vertices whose scala value is greater than Ver, the other is lower than Ver
    above_threshold = []
    below_threshold = []
    for v in VV:
        if v > Ver:
            above_threshold.append(v)
        else: # v < Ver, v is unique so v will never be equal to Ver
            below_threshold.append(v)
    
    # filter TE to only store edges that are incident in Ver
    edges = []
    for i in range(len(TE)):
        for e in TE[i][1]: # TE[i][1] = [e1, e2, e3, e4, e5, e6]
            if Ver in e:
                edges.append(e)
    
    def find_components(vertices):
        uf = UnionFind(len(vertices))
        vertex_to_index = {vertex: i for i, vertex in enumerate(vertices)}
        for v1, v2 in edges:
            if v1 in vertex_to_index and v2 in vertex_to_index:
                uf.union(vertex_to_index[v1], vertex_to_index[v2])
        component_map = {}
        for vertex in vertices:
            root = uf.find(vertex_to_index[vertex])
            if root not in component_map:
                component_map[root] = []
            component_map[root].append(vertex)
        return list(component_map.values())
    
    # upper_list is a list of lists, which stores all vertices in the link of Ver; those vertices have higher scalar values 
    upper_list = find_components(above_threshold)
    lower_list = find_components(below_threshold)
    
    # the current Ver is a maximum
    if len(upper_list) == 0:
        crit_type = 'maximum'
        return crit_type
        
    # the current Ver is a minimum
    elif len(lower_list) == 0:
        crit_type = 'minimum'
        return crit_type
    
    # the current Ver is a 1-saddle
    elif len(upper_list) == 1 and len(lower_list) == 1:
        crit_type = 'regular'
        return crit_type
    
    # the current Ver is a 1-saddle
    elif len(upper_list) == 2 and len(lower_list) == 1:
        crit_type = '1-saddle'
        return crit_type
    
    # the current Ver is a 2-saddle
    elif len(upper_list) == 1 and len(lower_list) == 2:
        crit_type = '2-saddle'
        return crit_type
    
    else:
        crit_type = 'monkey-saddle'
        return crit_type

compute_crit_pts_udf = udf(compute_crit_pts, StringType())

df_Crit = df_Crit_rela.withColumn("Crit_Type", compute_crit_pts_udf(df_Crit_rela.Ver, df_Crit_rela.VV, df_Crit_rela.TE_final))
df_Crit.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- TE_final: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |-- BoundaryOrNot: boolean (nullable = true)
 |-- Crit_Type: string (nullable = true)



In [29]:
t0 = time.time()

num_row = df_Crit.count()

t1 = time.time()
t_count_Crit = t1 - t0
print("Time cost:", t_count_Crit)
print("number of rows:", num_row)

Time cost: 3881.167540073395
number of rows: 1792989718


In [30]:
print("****************************")
print("Time cost for preparing VT:", t_count_VT)
print("Time cost for preparing VF:", t_count_VF)
print("Time cost for preparing VV:", t_count_VV)
print("Time cost for preparing TE:", t_count_TE_expected)

print("Time cost for concatenate the relations:", t_count_Crit_rela)
print("Total time cost for preparing the relations:", t_count_VT + t_count_VF + t_count_VV + t_count_TE_expected + t_count_Crit_rela)

print("****************************")
print("Time cost for calculating the critical points:", t_count_Crit)

print("****************************")
print("Total time cost:", t_count_VT + t_count_VF + t_count_VV + t_count_TE_expected + t_count_Crit_rela + t_count_Crit)

****************************
Time cost for preparing VT: 501.4889419078827
Time cost for preparing VF: 460.74542474746704
Time cost for preparing VV: 455.9120762348175
Time cost for preparing TE: 2889.5364854335785
Time cost for concatenate the relations: 3812.731728553772
Total time cost for preparing the relations: 8120.414656877518
****************************
Time cost for calculating the critical points: 3881.167540073395
****************************
Total time cost: 12001.582196950912
